In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
from pathlib import Path
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.processors import build_processor
from multiplex_pipeline.processors.controller import ResourceBuildingController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\BLCA-2_Analysis_todel\analysis_settings_BLCA2_todel.yaml'

overwrite_mask = True

settings = load_analysis_settings(settings_path)
settings

{'image_dir': 'R:/CellDive/BLCA-2/BLCA-2_Final',
 'analysis_name': 'BLCA-2_Analysis_todel',
 'local_analysis_dir': 'C:/',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('C:/BLCA-2_Analysis_todel/logs'),
 'detection_image': 'BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-2_Analysis_todel/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-2_Analysis_todel/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-2_Analysis_todel/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': 'DAPI',
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'additional_elements': [{'category': 'image_transfo

### Define the logger

In [3]:
log_file = settings['log_dir'] / f"cores_segmentation_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

2

### Define cores for the analysis

In [4]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

[WindowsPath('C:/BLCA-2_Analysis_todel/cores/Core_000.zarr')]

### Setup

In [5]:
pyramid_levels = settings['sdata_storage']['max_pyramid_level']
downscale = settings['sdata_storage']['downscale']
chunk_size = settings['sdata_storage']['chunk_size']

# setup builders of additional data elements

if settings.get('additional_elements', None):
    
    builders_list = []

    for builder_settings in settings['additional_elements']:
        
        params = builder_settings.get("parameters") or {}
        print(builder_settings)

        builder = build_processor(builder_settings['category'], builder_settings['type'], **params) 
        
        builder_controller = ResourceBuildingController(builder=builder, 
                                            input_names=builder_settings['input'], 
                                            output_names=builder_settings['output'], 
                                            keep=builder_settings.get('keep', False), 
                                            overwrite=True,
                                            pyramid_levels=pyramid_levels,
                                            downscale = downscale,
                                            chunk_size = chunk_size,
                                            )
        
        logger.info(f"Image transformer of type '{builder_settings['type']}' for image '{builder_settings['input']}' has been created.")

        builders_list.append(builder_controller)

else:
    builders_list = []
    logger.info("No resource builders specified.")

{'category': 'image_transformer', 'type': 'normalize', 'input': 'DAPI', 'output': 'DAPI_norm', 'parameters': {'low': 1, 'high': 99.5}, 'keep': False}
2025-10-14 16:09:51.819 | INFO     | __main__:<module>:28 - Image transformer of type 'normalize' for image 'DAPI' has been created.
{'category': 'object_segmenter', 'type': 'instanseg', 'parameters': {'model': 'fluorescence_nuclei_and_cells', 'pixel_size': 0.3, 'resolve_cell_and_nucleus': True, 'cleanup_fragments': True}, 'input': ['DAPI_norm'], 'output': ['instanseg_nucleus_org', 'instanseg_cell_org']}
Model fluorescence_nuclei_and_cells version 0.1.0 already downloaded in c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda
2025-10-14 16:09:52.620 | INFO     | __main__:<module>:28 - Image transformer of type 'instanseg' for image '['DAPI_norm']' has been created.
{'category': 'mask_builder', 'type': 'blob', 'input': 'instanseg_nucleus_or

### Processing

In [23]:
for sd_path in path_list:
    
    print(f"Processing {sd_path.name}")

    # get sdata
    sdata = sd.read_zarr(sd_path)

    # run builders of additional elements
    for builder_controller in builders_list:
        sdata = builder_controller.run(sdata)

version mismatch: detected: RasterFormatV02, requested: FormatV04


Processing Core_000.zarr


c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


2025-10-14 16:26:54.372 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:51 - All channels have required resolution level: 0
2025-10-14 16:26:55.361 | INFO     | multiplex_pipeline.processors.image_transformers:run:73 - Applied normalization (percentiles 1.0–99.5) → [98.0, 3871.0]
2025-10-14 16:26:55.364 | INFO     | multiplex_pipeline.processors.controller:run:136 - New element(s) '['DAPI_norm']' have been created.
2025-10-14 16:26:55.534 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:51 - All channels have required resolution level: 0


2025-10-14 16:27:13.787 | INFO     | multiplex_pipeline.processors.controller:run:136 - New element(s) '['instanseg_nucleus_org', 'instanseg_cell_org']' have been created.
2025-10-14 16:27:16.146 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:51 - All channels have required resolution level: 0
2025-10-14 16:27:16.147 | WARNING  | multiplex_pipeline.processors.controller:prepare_to_overwrite:72 - Mask name 'blob' already exists and will be overwritten.
2025-10-14 16:27:16.535 | INFO     | multiplex_pipeline.processors.controller:prepare_to_overwrite:77 - Existing mask 'blob' deleted from sdata.
2025-10-14 16:27:20.950 | INFO     | multiplex_pipeline.processors.controller:run:136 - New element(s) '['blob']' have been created.
2025-10-14 16:27:22.445 | INFO     | multiplex_pipeline.processors.controller:run:154 - Mask 'blob' has been saved to disk.
2025-10-14 16:27:22.465 | INFO     | multiplex_pipeline.processors.controller:validate_resolution_present:5

### Sneak peak

In [15]:
# refresh the object
sdata_org = sd.read_zarr(path_list[0])
sdata_org

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


SpatialData object, with associated Zarr store: C:\BLCA-2_Analysis_todel\cores\Core_000.zarr
├── Images
│     └── 'DAPI': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784)
├── Labels
│     ├── 'blob': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
│     ├── 'cytoplasm': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
│     ├── 'instanseg_cell': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
│     ├── 'instanseg_nucleus': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
│     └── 'ring': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
└── Tables
      ├── 'instanseg_data': AnnData (11394, 6)
      └── 'instanseg_table': AnnData (11394, 4)
with coordinate systems:
    ▸ 'global', with elements:
        DAPI (Images), blob (Labels), cytoplasm (Labels), instanseg_cell (Labels), instanseg_nucleus (Labels), ring (Labels)

In [ ]:
Interactive(sdata_org)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


2025-10-14 16:28:08.716 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:08.717 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:15.641 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:15.652 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:37.324 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:37.331 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:52.705 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2025-10-14 16:28:52.710 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
